In [37]:
import requests
import time
import os
from bs4 import BeautifulSoup
import itertools
import pandas
proxies = {'http': 'http://proxy.charite.de:8080',
           'https': 'https://proxy.charite.de:8080'}
print("Using HTTP proxy %s" % proxies['http'])

Using HTTP proxy http://proxy.charite.de:8080


In [38]:
# get the list
list_PIs = []
with open('PIs.txt') as fin:
    for line in fin:
        list_PIs.append(line)

In [39]:
# print one item of list
print(list_PIs[10])

# add a plus between name and last name
for ll in range(0,len(list_PIs)-1):
    list_PIs[ll] = list_PIs[ll].replace(' ','+')
    list_PIs[ll] = list_PIs[ll].replace('\n','')
    list_PIs[ll] = list_PIs[ll] + '[author]'


Nina Romanczuk-Seiferth



In [40]:
print(len(list_PIs))

40


In [41]:
# get combinations of authors
comb_list = list(itertools.combinations(list_PIs,2))
print(len(comb_list))

780


In [42]:
# make the query string
query_strings = []
for counter, ll in enumerate(comb_list):
    query_strings.append(ll[0] + '+AND+' + ll[1])
    
print(query_strings[30])


Christine+Heim[author]+AND+Christian+Beste[author]


In [43]:
# make th url data base query
url_queries = []

for counter, ll in enumerate(query_strings):
    url_queries.append('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=' + ll)
    
print(url_queries[30])



https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Christine+Heim[author]+AND+Christian+Beste[author]


In [44]:
# call all urls and get the number of results
all_counts = []
all_papers = []
for counter, curURL in enumerate(url_queries):
    s      = requests.session()
    r      = s.get(curURL,proxies=proxies)
    soup   = BeautifulSoup(r.text,"html5lib")
    
    # get the counts
    counts = soup.find_all("count")
    print(counter)
    if len(counts) == 0:
        print(soup)
        break
    all_counts.append(counts[0].get_text())
    time.sleep(1)
    
    # get the papers
    papers = soup.find_all("id")
    for counter, cur_paper in enumerate(papers):
        all_papers.append(papers[counter].get_text())
    

    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [45]:
# check result
#print(query_strings[0:5])
#print(all_counts)
#print(all_papers)

#print(comb_list[0])

# get the unique joint papers
unique_papers = set(all_papers)

print(unique_papers)

# print the number of joint publications
print(' ')
print('We have ' + str(len(unique_papers)) + ' joint publications.')
print('We have ' + str(len(all_papers)) + ' nonunique publications.')

{'18703319', '27374936', '25582269', '12645817', '29169660', '28798371', '22767373', '28671956', '30302403', '28740821', '30410047', '24756342', '27974817', '27909319', '30249471', '20673876', '25035082', '24470632', '29662068', '26433217', '30268001', '21790907', '27810934', '27086240', '29058369', '28491495', '27791105', '25566131', '24642287', '24060832', '28174031', '26982326', '30269148', '27415603', '25833034', '21309952', '28207073', '27079174', '26463560', '21262256', '26258487', '19581569', '17488322', '28649313', '12555235', '20570115', '23466521', '24254424', '29860087', '18606955', '23679679', '23564736', '22330316', '19490027', '27751654', '24283978', '25526597', '24176673', '27697987', '28589968', '26573969', '30583245', '24802075', '30111185', '24166646', '27694318', '29953935', '29656430', '26276438', '26900791', '27031567', '16899037', '14702246', '27145766', '27300740', '24079538', '25639749', '22520437', '27405334', '29050056', '24265700', '25913102', '25819024', '24

In [46]:
# define function for cleaning string
def clean_string(cur_str):
    cur_str = cur_str.replace('[author]','')
    cur_str = cur_str.replace('+',' ')
    return(cur_str)

# prepare columns for output file
first_author  = []
second_author = []

print(clean_string(comb_list[2][1]))
for counter, item in enumerate(comb_list):
    first_author.append(clean_string(comb_list[counter][0]))
    second_author.append(clean_string(comb_list[counter][1]))
    
    

Florian Schlagenhauf


In [47]:
# make a data frame ready for printing
df = pandas.DataFrame({'FirstAuthor': first_author, 'SecondAuthor': second_author,
                      'common_pubs': all_counts})
df

writer = pandas.ExcelWriter('wermitwem.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()


In [103]:
# print(soup.prettify()[0:100000])

<!--?xml version="1.0" encoding="UTF-8" ?-->
<!DOCTYPE esearchresult PUBLIC "-//NLM//DTD esearch 20060628//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">
<html>
 <head>
 </head>
 <body>
  <esearchresult>
   <count>
    50
   </count>
   <retmax>
    20
   </retmax>
   <retstart>
    0
   </retstart>
   <idlist>
    <id>
     30217552
    </id>
    <id>
     29847018
    </id>
    <id>
     29744956
    </id>
    <id>
     29313106
    </id>
    <id>
     29058369
    </id>
    <id>
     28673442
    </id>
    <id>
     28491495
    </id>
    <id>
     28317230
    </id>
    <id>
     27645501
    </id>
    <id>
     27726253
    </id>
    <id>
     27628616
    </id>
    <id>
     27480622
    </id>
    <id>
     27397865
    </id>
    <id>
     27145766
    </id>
    <id>
     27130821
    </id>
    <id>
     27096667
    </id>
    <id>
     26435383
    </id>
    <id>
     26289945
    </id>
    <id>
     26048580
    </id>
    <id>
     25988224
    </id>
   

In [110]:
print(counts[0].get_text())

50


In [27]:
contacts = {}
for element in links:
    contacts[element.a.get_text()] = {}
first_keys = list(contacts.keys())[0:9]
for key in first_keys:
    print(key)
    print(contacts[key])

Netzwerk SHB
{}
Psychotherapeutische Praxis - Karl-Joachim Bayer
{}
Vivantes Humboldt-Klinikum
{}
Psychotherapeutische Praxis - Götz Saecker
{}
Psychotherapeutische Praxis - Ulrike Sommer-Stratmann
{}
Psychotherapeutische Praxis - Claudia Sautier
{}
Psychotherapeutische Praxis - Susanne Raetzer
{}
Glücksspiel-Sucht-Hilfe e.V.
{}
Psychotherapeutische Praxis - Sylvia Nawothnig
{}


In [30]:
print(links[0].a["href"])
prefix = 'http://www.spielsucht-hilfe-berlin.de'

/shb/index.php?title=AG_Spielsucht


In [31]:
for element in links:
    contacts[element.a.get_text()]["link"] = prefix + element.a["href"]

In [32]:
for key in first_keys:
    print(key)
    print(contacts[key])

Netzwerk SHB
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Netzwerk_Spielsucht-Hilfe-Berlin'}
Psychotherapeutische Praxis - Karl-Joachim Bayer
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_Karl-Joachim_Bayer'}
Vivantes Humboldt-Klinikum
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Vivantes_Humboldt-Klinikum'}
Psychotherapeutische Praxis - Götz Saecker
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_G%C3%B6tz_Saecker'}
Psychotherapeutische Praxis - Ulrike Sommer-Stratmann
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_Ulrike_Sommer-Stratmann'}
Psychotherapeutische Praxis - Claudia Sautier
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_Claudia_Sautier'}
Psychotherapeutische Praxis - Susanne Raetzer
{'link': 'http://www.spielsucht-hilfe-berlin.

In [50]:
# now go the link and apply a function that reads out all the information
# do it with first one first
key = list(contacts.keys())[0:2][1]
#for key in first_keys:
print(key)
print(contacts[key])
# get the link
curLink = contacts[key]['link']
print(curLink)

s      = requests.session()
r      = s.get(curLink,proxies=proxies)
ssoup  = BeautifulSoup(r.text,"html5lib")
print(type(ssoup))
print(ssoup.prettify()[0:1000000])

Psychotherapeutische Praxis - Karl-Joachim Bayer
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_Karl-Joachim_Bayer'}
http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_Karl-Joachim_Bayer
<class 'bs4.BeautifulSoup'>
<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="de">
 <head>
  <meta charset="utf-8"/>
  <title>
   Psychotherapeutische Praxis - Karl-Joachim Bayer – Spielsucht-Hilfe-Berlin
  </title>
  <meta content="MediaWiki 1.25.2" name="generator"/>
  <link href="/favicon.ico" rel="shortcut icon"/>
  <link href="/shb/opensearch_desc.php" rel="search" title="Spielsucht-Hilfe-Berlin (de)" type="application/opensearchdescription+xml"/>
  <link href="http://spielsucht-hilfe-berlin.de/shb/api.php?action=rsd" rel="EditURI" type="application/rsd+xml"/>
  <link href="/shb/index.php?title=Psychotherapeutische_Praxis_-_Karl-Joachim_Bayer" hreflang="x-default" rel="alternate"/>
  <link href="/shb/in

In [58]:
infos = ssoup.find_all("td")
print (type(infos))
print(len(infos))
print(infos[0:3])

<class 'bs4.element.ResultSet'>
11
[<td colspan="2" style="width:66%; font-weight:bold;">Dipl.-Psych. Karl-Joachim Bayer
</td>, <td colspan="1" rowspan="10" style="padding:0.5em 0.4em; text-align:center; vertical-align:top;">Psychologischer Psychotherapeut
<p><b>Leistungen für Erwachsene bei privater Kostenübernahme:</b>
</p>
<ul><li>Familientherapie</li>
<li>Krisenintervention</li>
<li>Paartherapie</li>
<li>Systemische Therapie</li></ul>
</td>, <td colspan="2" style="padding:0.2em 0.4em; margin:0 0 10px 0; background:#cccccc; font-weight:bold;">Adresse:
</td>]


In [57]:
infoList = {}
for element in infos:
    infoList[element.a.get_text()] = {}

AttributeError: 'NoneType' object has no attribute 'get_text'